In [1]:
# default_exp inference.translation

# Translation
> Translation API for AdaptNLP

In [2]:
#hide
from fastcore.test import test_eq
from nbverbose.showdoc import *

In [16]:
#export
import logging
from typing import List, Dict, Union
from collections import defaultdict, OrderedDict

import torch
from torch.utils.data import TensorDataset, DataLoader

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    PreTrainedTokenizer,
    PreTrainedModel,
    T5ForConditionalGeneration,
)

from adaptnlp.model import AdaptiveModel
from adaptnlp.callback import GeneratorCallback

from fastai.torch_core import apply, to_device

from fastcore.basics import Self
from adaptnlp.model_hub import HFModelResult, FlairModelResult, HFModelHub, FlairModelHub
from adaptnlp.result import DetailLevel

In [17]:
#export
logger = logging.getLogger(__name__)

In [18]:
#export
class TranslationResult:
    "A basic result class for Translation problems"
    def __init__(
        self,
        inputs:List[str], # A list of input string sentences
        input_lang:str, # A input language
        output_lang:str, # An output language
        translations:List[str] # A list of the translated sentences
    ):
        self.inputs = inputs
        self.input_lang = input_lang
        self.output_lang = output_lang
        self.translations = translations
    
    def to_dict(
        self, 
        detail_level:DetailLevel=DetailLevel.Low # A detail level to return
    ):
        "Convert `self` to a filtered dictionary"
        o = OrderedDict()
        o.update(
            {'translations':self.translations,}
        )
        if detail_level == 'medium' or detail_level == 'high':
            o.update({
                'inputs': self.inputs,
                'input_language': self.input_lang,
                'output_language': self.output_lang
            })
        if detail_level == 'high':
            print("Warning: There is no difference between `medium` and `high` for Translation tasks")
        return o

In [19]:
show_doc(TranslationResult.to_dict)

<h4 id="TranslationResult.to_dict" class="doc_header"><code>TranslationResult.to_dict</code><a href="__main__.py#L16" class="source_link" style="float:right">[source]</a></h4>

> <code>TranslationResult.to_dict</code>(**`detail_level`**:`DetailLevel`=*`'low'`*)

Convert `self` to a filtered dictionary

**Parameters:**


 - **`detail_level`** : *`<class 'fastcore.basics.DetailLevel'>`*, *optional*	<p>A detail level to return</p>



In [26]:
#export
class TransformersTranslator(AdaptiveModel):
    "Adaptive model for Transformer's Conditional Generation or Language Models (Transformer's T5 and Bart conditional generation models have a language modeling head)"

    def __init__(
        self, 
        tokenizer: PreTrainedTokenizer, # A tokenizer object from Huggingface's transformers (TODO)and tokenizers
        model: PreTrainedModel # A transformers Conditional Generation (Bart or T5) or Language model
    ):
        # Load up model and tokenizer
        self.tokenizer = tokenizer
        super().__init__()
        
        # Sets internal model
        self.set_model(model)
        
        # Set inputs to come in as `dict`
        super().set_as_dict(True)

    @classmethod
    def load(
        cls, 
        model_name_or_path: str # A key string of one of Transformer's pre-trained translator Model
    ) -> AdaptiveModel:
        "Class method for loading and constructing this classifier"
        tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
        translator = cls(tokenizer, model)
        return translator

    def predict(
        self,
        text: Union[List[str], str], # Sentences to run inference on
        t5_prefix: str = 'translate English to German', # The pre-appended prefix for the specificied task. Only in use for T5-type models.
        mini_batch_size: int = 32,  # Mini batch size
        num_beams: int = 1, # Number of beams for beam search. Must be between 1 and infinity. 1 means no beam search
        min_length: int = 0, # The min length of the sequence to be generated
        max_length: int = 128, # The max length of the sequence to be generated. Between min_length and infinity
        early_stopping: bool = True, # If set to `True` beam search is stopped when at least num_beams sentences finished per batch
        detail_level:DetailLevel = DetailLevel.Low, # The level of detail to return
        **kwargs, # Optional arguments for the Transformers `PreTrainedModel.generate()` method
    ) -> List[str]: # A list of translated sentences
        "Predict method for running inference using the pre-trained sequence classifier model. Keyword arguments for parameters of the method `Transformers.PreTrainedModel.generate()` can be used as well"
        
        # Make all inputs lists
        if isinstance(text, str):
            text = [text]
        
        # T5 requires 'translate: ' precursor text for pre-trained translator
        if isinstance(self.model, T5ForConditionalGeneration):
            text = [f'{t5_prefix}: {t}' for t in text]
            
        dataset = self._tokenize(text)
        dl = DataLoader(dataset, batch_size=mini_batch_size)
        translations = []
        
        logger.info(f'Running translator on {len(dataset)} text sequences')
        logger.info(f'Batch size = {mini_batch_size}')
        
        cb = GeneratorCallback(num_beams, min_length, max_length, early_stopping, **kwargs)
        
        preds,_ = super().get_preds(dl=dl, cbs=[cb])
        
        preds = apply(Self.squeeze(0), preds)

        for o in preds:
            translations.append(
                [
                    self.tokenizer.decode(
                        o,
                        skip_special_tokens=True,
                        clean_up_tokenization_spaces=False,
                    )
                ].pop()
            )
        
        languages = t5_prefix.strip('translate ').split(' to ')
        
        res = TranslationResult(text, *languages, translations)

        return res if detail_level is None else res.to_dict(detail_level)

    def _tokenize(self, text: Union[List[str], str]) -> TensorDataset:
        """ Batch tokenizes text and produces a `TensorDataset` with text """

        tokenized_text = self.tokenizer.batch_encode_plus(
            text,
            return_tensors='pt',
            max_length=512,
            padding='max_length',
            add_special_tokens=True,
        )

        # Bart doesn't use `token_type_ids`
        dataset = TensorDataset(
            tokenized_text['input_ids'],
            tokenized_text['attention_mask'],
        )
        return dataset

In [27]:
show_doc(TransformersTranslator.load)

<h4 id="TransformersTranslator.load" class="doc_header"><code>TransformersTranslator.load</code><a href="__main__.py#L20" class="source_link" style="float:right">[source]</a></h4>

> <code>TransformersTranslator.load</code>(**`model_name_or_path`**:`str`)

Class method for loading and constructing this classifier

**Parameters:**


 - **`model_name_or_path`** : *`<class 'str'>`*	<p>A key string of one of Transformer's pre-trained translator Model</p>



**Returns**:
	
 * *`<class 'adaptnlp.model.AdaptiveModel'>`*

In [22]:
show_doc(TransformersTranslator.predict)

<h4 id="TransformersTranslator.predict" class="doc_header"><code>TransformersTranslator.predict</code><a href="__main__.py#L40" class="source_link" style="float:right">[source]</a></h4>

> <code>TransformersTranslator.predict</code>(**`text`**:`Union`\[`List`\[`str`\], `str`\], **`t5_prefix`**:`str`=*`'translate English to German'`*, **`mini_batch_size`**:`int`=*`32`*, **`num_beams`**:`int`=*`1`*, **`min_length`**:`int`=*`0`*, **`max_length`**:`int`=*`128`*, **`early_stopping`**:`bool`=*`True`*, **`detail_level`**:`DetailLevel`=*`'low'`*, **\*\*`kwargs`**)

Predict method for running inference using the pre-trained sequence classifier model. Keyword arguments for parameters of the method `Transformers.PreTrainedModel.generate()` can be used as well

**Parameters:**


 - **`text`** : *`typing.Union[typing.List[str], str]`*	<p>Sentences to run inference on</p>


 - **`t5_prefix`** : *`<class 'str'>`*, *optional*	<p>The pre-appended prefix for the specificied task. Only in use for T5-type models.</p>


 - **`mini_batch_size`** : *`<class 'int'>`*, *optional*	<p>Mini batch size</p>


 - **`num_beams`** : *`<class 'int'>`*, *optional*	<p>Number of beams for beam search. Must be between 1 and infinity. 1 means no beam search</p>


 - **`min_length`** : *`<class 'int'>`*, *optional*	<p>The min length of the sequence to be generated</p>


 - **`max_length`** : *`<class 'int'>`*, *optional*	<p>The max length of the sequence to be generated. Between min_length and infinity</p>


 - **`early_stopping`** : *`<class 'bool'>`*, *optional*	<p>If set to `True` beam search is stopped when at least num_beams sentences finished per batch</p>


 - **`detail_level`** : *`<class 'fastcore.basics.DetailLevel'>`*, *optional*	<p>The level of detail to return</p>


 - **`kwargs`** : *`<class 'inspect._empty'>`*


**Returns**:
	
 * *`typing.List[str]`*	<p>A list of translated sentences</p>



In [23]:
#export
class EasyTranslator:
    "Translation Module"

    def __init__(self):
        self.translators: Dict[AdaptiveModel] = defaultdict(bool)

    def translate(
        self,
        text: Union[List[str], str], # Sentences to run inference on
        model_name_or_path: str = "t5-small", # A model id or path to a pre-trained model repository or custom trained model directory
        t5_prefix: str = "translate English to German", # The pre-appended prefix for the specificied task. Only in use for T5-type models
        detail_level=DetailLevel.Low, # The level of detail to return
        mini_batch_size: int = 32, # Mini batch size
        num_beams: int = 1, # Number of beams for beam search. Must be between 1 and infinity. 1 means no beam search
        min_length: int = 0, # The min length of the sequence to be generated
        max_length: int = 128, # The max length of the sequence to be generated. Between min_length and infinity
        early_stopping: bool = True, # If set to `True` beam search is stopped when at least num_beams sentences finished per batch
        **kwargs, # Optional arguments for the Transformers `PreTrainedModel.generate()` method
    ) -> List[str]: 
        "Predict method for running inference using the pre-trained sequence classifier model. Keyword arguments for parameters of the method `Transformers.PreTrainedModel.generate()` can be used as well."
        name = getattr(model_name_or_path, 'name', model_name_or_path)
        if not self.translators[name]:
            self.translators[name] = TransformersTranslator.load(
                name
            )

        translator = self.translators[name]
        return translator.predict(
            text=text,
            t5_prefix=t5_prefix,
            mini_batch_size=mini_batch_size,
            num_beams=num_beams,
            min_length=min_length,
            max_length=max_length,
            early_stopping=early_stopping,
            **kwargs,
        )

In [25]:
show_doc(EasyTranslator.translate)

<h4 id="EasyTranslator.translate" class="doc_header"><code>EasyTranslator.translate</code><a href="__main__.py#L8" class="source_link" style="float:right">[source]</a></h4>

> <code>EasyTranslator.translate</code>(**`text`**:`Union`\[`List`\[`str`\], `str`\], **`model_name_or_path`**:`str`=*`'t5-small'`*, **`t5_prefix`**:`str`=*`'translate English to German'`*, **`detail_level`**=*`'low'`*, **`mini_batch_size`**:`int`=*`32`*, **`num_beams`**:`int`=*`1`*, **`min_length`**:`int`=*`0`*, **`max_length`**:`int`=*`128`*, **`early_stopping`**:`bool`=*`True`*, **\*\*`kwargs`**)

Predict method for running inference using the pre-trained sequence classifier model. Keyword arguments for parameters of the method `Transformers.PreTrainedModel.generate()` can be used as well.

**Parameters:**


 - **`text`** : *`typing.Union[typing.List[str], str]`*	<p>Sentences to run inference on</p>


 - **`model_name_or_path`** : *`<class 'str'>`*, *optional*	<p>A model id or path to a pre-trained model repository or custom trained model directory</p>


 - **`t5_prefix`** : *`<class 'str'>`*, *optional*	<p>The pre-appended prefix for the specificied task. Only in use for T5-type models</p>


 - **`detail_level`** : *`<class 'str'>`*, *optional*	<p>The level of detail to return</p>


 - **`mini_batch_size`** : *`<class 'int'>`*, *optional*	<p>Mini batch size</p>


 - **`num_beams`** : *`<class 'int'>`*, *optional*	<p>Number of beams for beam search. Must be between 1 and infinity. 1 means no beam search</p>


 - **`min_length`** : *`<class 'int'>`*, *optional*	<p>The min length of the sequence to be generated</p>


 - **`max_length`** : *`<class 'int'>`*, *optional*	<p>The max length of the sequence to be generated. Between min_length and infinity</p>


 - **`early_stopping`** : *`<class 'bool'>`*, *optional*	<p>If set to `True` beam search is stopped when at least num_beams sentences finished per batch</p>


 - **`kwargs`** : *`<class 'inspect._empty'>`*


**Returns**:
	
 * *`typing.List[str]`*	<p>Optional arguments for the Transformers `PreTrainedModel.generate()` method</p>



In [24]:
#hide
text = ["Machine learning will take over the world very soon.",
        "Machines can speak in many languages.",]

translator = EasyTranslator()
translations = translator.translate(text = text, t5_prefix="translate English to German", model_name_or_path="t5-small", mini_batch_size=1, min_length=0, max_length=100, early_stopping=True)
test_eq(translations['translations'], ['Das Maschinenlernen wird die Welt in Kürze übernehmen.',
 'Maschinen können in vielen Sprachen sprechen.'])

In [ ]:
#hide
hub = HFModelHub()
model = hub.search_model_by_task("translation")[-1]
translations = translator.translate(text = text, t5_prefix="translate English to German", model_name_or_path=model, mini_batch_size=1, min_length=0, max_length=100, early_stopping=True)
test_eq(translations['translations'], ['Das Maschinenlernen wird die Welt in Kürze übernehmen.',
 'Maschinen können in vielen Sprachen sprechen.'])